In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')


zip_path = "/content/drive/MyDrive/Deep-learning Project/Kathbath-Bengali-Test-Known.zip"
extract_path = "/content/dataset"


os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted to:", extract_path)
print("📂 Files:", os.listdir(extract_path))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dataset extracted to: /content/dataset
📂 Files: ['Kathbath-Bengali-Test-Known']


In [11]:

import os

DATA_PATH = "/content/dataset/Kathbath-Bengali-Test-Known"
for root, dirs, files in os.walk(DATA_PATH):
    print(f"📁 {root} → {len(files)} files, {len(dirs)} subfolders")
    if any(f.endswith('.json') or f.endswith('.csv') or f.endswith('.tsv') for f in files):
        print("🗃️", [f for f in files if f.endswith(('.json','.csv','.tsv'))][:5])
    break


📁 /content/dataset/Kathbath-Bengali-Test-Known → 2 files, 1 subfolders
🗃️ ['data.json', 'params.json']


In [12]:
import json, os

json_path = "/content/dataset/Kathbath-Bengali-Test-Known/data.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print("🔹 Type of JSON:", type(data))
print("🔹 Length:", len(data))


if isinstance(data, list):
    print("🔹 Keys in first entry:", data[0].keys())
    print("🔹 Example entry:\n", data[0])
elif isinstance(data, dict):
    first_key = list(data.keys())[0]
    print("🔹 Keys in first entry:", data[first_key].keys())
    print("🔹 Example entry:\n", data[first_key])


🔹 Type of JSON: <class 'list'>
🔹 Length: 2806
🔹 Keys in first entry: dict_keys(['audioFilename', 'text', 'gender', 'speaker'])
🔹 Example entry:
 {'audioFilename': 'audios/844424931171856-711-f.wav', 'text': 'শেখ হাসিনাকে হত্যাচেষ্টা মামলায় এগার জনের বিশ বছর করে কারাদণ্ড', 'gender': 'female', 'speaker': '711'}


In [13]:
import os, json, csv

DATA_PATH = "/content/dataset/Kathbath-Bengali-Test-Known"
json_path = os.path.join(DATA_PATH, "data.json")
csv_path = os.path.join(DATA_PATH, "metadata.csv")


with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Created metadata.csv
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["utt_id", "audio_filepath", "transcript", "gender", "speaker"])

    for i, entry in enumerate(data):
        audio_file = os.path.join(DATA_PATH, entry["audioFilename"])
        transcript = entry["text"]
        gender = entry["gender"]
        speaker = entry["speaker"]
        utt_id = f"utt_{i:05d}"
        writer.writerow([utt_id, audio_file, transcript, gender, speaker])

print(f"✅ metadata.csv created successfully → {csv_path}")


✅ metadata.csv created successfully → /content/dataset/Kathbath-Bengali-Test-Known/metadata.csv


In [14]:
import pandas as pd
meta = pd.read_csv(csv_path)
print("Rows:", len(meta))
meta.head()


Rows: 2806


,utt_id,audio_filepath,transcript,gender,speaker
0,utt_00000,/content/dataset/Kathbath-Bengali-Test-Known/a...,শেখ হাসিনাকে হত্যাচেষ্টা মামলায় এগার জনের বিশ...,female,711
1,utt_00001,/content/dataset/Kathbath-Bengali-Test-Known/a...,আমরা দুজন ক্রমশ দূরে চলে যাওয়া দৃশ্যের পর দৃশ...,female,858
2,utt_00002,/content/dataset/Kathbath-Bengali-Test-Known/a...,আত্মসমর্পণ করার জন্য ত্রিশ দিন সময় চাইলেন সজ্...,female,858
3,utt_00003,/content/dataset/Kathbath-Bengali-Test-Known/a...,ঢাকায় ভাতের মধ্যে একটি চুল পেয়ে শিশু গৃহকর্ম...,female,1037
4,utt_00004,/content/dataset/Kathbath-Bengali-Test-Known/a...,প্রচ্ছদ জাতীয় শেরপুরে আওয়ামীলীগের মনোনয়ন প্...,female,428


In [15]:
!pip install torch torchaudio librosa soundfile


In [16]:
!pip install jiwer


In [4]:
!pip install --quiet torch torchvision torchaudio librosa soundfile jiwer tensorboard


In [2]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA available: True
GPU name: Tesla T4


In [5]:
import os, json, csv, math, random, time
from pathlib import Path
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from jiwer import wer
from torch.cuda.amp import autocast, GradScaler

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)


✅ Using device: cuda


In [17]:
import os, json, csv, math, random, time
from pathlib import Path
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from jiwer import wer
from torch.cuda.amp import autocast, GradScaler


DATA_PATH = "/content/dataset/Kathbath-Bengali-Test-Known"
METADATA_CSV = os.path.join(DATA_PATH, "metadata.csv")
MODEL_DIR = "/content/models"
os.makedirs(MODEL_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [18]:

if not os.path.exists(METADATA_CSV):
    json_path = os.path.join(DATA_PATH, "data.json")
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    with open(METADATA_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["utt_id", "audio_filepath", "transcript", "gender", "speaker"])
        for i, entry in enumerate(data):
            audio_file = os.path.join(DATA_PATH, entry["audioFilename"])
            transcript = entry["text"]
            gender = entry.get("gender", "")
            speaker = entry.get("speaker", "")
            writer.writerow([f"utt_{i:05d}", audio_file, transcript, gender, speaker])
    print("Wrote metadata.csv")

meta = pd.read_csv(METADATA_CSV)
print("Total samples:", len(meta))
meta.head()


Total samples: 2806


,utt_id,audio_filepath,transcript,gender,speaker
0,utt_00000,/content/dataset/Kathbath-Bengali-Test-Known/a...,শেখ হাসিনাকে হত্যাচেষ্টা মামলায় এগার জনের বিশ...,female,711
1,utt_00001,/content/dataset/Kathbath-Bengali-Test-Known/a...,আমরা দুজন ক্রমশ দূরে চলে যাওয়া দৃশ্যের পর দৃশ...,female,858
2,utt_00002,/content/dataset/Kathbath-Bengali-Test-Known/a...,আত্মসমর্পণ করার জন্য ত্রিশ দিন সময় চাইলেন সজ্...,female,858
3,utt_00003,/content/dataset/Kathbath-Bengali-Test-Known/a...,ঢাকায় ভাতের মধ্যে একটি চুল পেয়ে শিশু গৃহকর্ম...,female,1037
4,utt_00004,/content/dataset/Kathbath-Bengali-Test-Known/a...,প্রচ্ছদ জাতীয় শেরপুরে আওয়ামীলীগের মনোনয়ন প্...,female,428


In [20]:

SAMPLE_RATE = 16000
N_MELS = 80
N_FFT = 1024
HOP_LENGTH = 256
WIN_LENGTH = 1024

def load_audio(path, sr=SAMPLE_RATE):
    wav, orig_sr = sf.read(path)
    if len(wav.shape) > 1:
        wav = np.mean(wav, axis=1)
    if orig_sr != sr:
        wav = librosa.resample(wav.astype(np.float32), orig_sr, sr)
    return wav.astype(np.float32)

def compute_log_mel(wav):
    mel = librosa.feature.melspectrogram(y=wav, sr=SAMPLE_RATE, n_fft=N_FFT,
                                         hop_length=HOP_LENGTH, win_length=WIN_LENGTH, n_mels=N_MELS, power=1.0)
    log_mel = librosa.power_to_db(mel)

    log_mel = (log_mel - log_mel.mean()) / (log_mel.std() + 1e-9)
    return log_mel.astype(np.float32)

class BengaliASRDataset(Dataset):
    def __init__(self, metadata_df, max_duration=None):
        self.df = metadata_df.reset_index(drop=True)
        if max_duration:

            pass
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        audio_path = row['audio_filepath']
        transcript = str(row['transcript'])
        wav = load_audio(audio_path)
        feat = compute_log_mel(wav)

        feat = torch.from_numpy(feat).unsqueeze(0)
        target = torch.tensor(encode_text(transcript), dtype=torch.long)
        return feat, target

def collate_fn(batch):

    feats = [b[0].squeeze(0).transpose(0,1) for b in batch]
    feat_lens = torch.tensor([f.shape[0] for f in feats], dtype=torch.long)
    feats_padded = pad_sequence(feats, batch_first=True)
    feats_padded = feats_padded.transpose(1,2).unsqueeze(1)
    targets = [b[1] for b in batch]
    targets_concat = torch.cat(targets)
    target_lens = torch.tensor([len(t) for t in targets], dtype=torch.long)
    return feats_padded, feat_lens, torch.cat(targets), target_lens, targets


In [21]:
dataset = BengaliASRDataset(meta.sample(frac=1.0).reset_index(drop=True))
loader = DataLoader(dataset, batch_size=4, collate_fn=collate_fn, num_workers=2)
feats_padded, feat_lens, targets_concat, target_lens, targets_list = next(iter(loader))
print("feats:", feats_padded.shape, "feat_lens:", feat_lens, "targets_concat_len:", targets_concat.shape)


feats: torch.Size([4, 1, 80, 501]) feat_lens: tensor([366, 340, 286, 501]) targets_concat_len: torch.Size([242])


In [22]:
class ConvBlock(nn.Module):
    def __init__(self, in_ch=1, out_ch=128):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_ch)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        return x

class CNNRNN_ASR(nn.Module):
    def __init__(self, n_mels=80, cnn_channels=128, lstm_hidden=512, lstm_layers=2, n_classes=100):
        super().__init__()
        self.conv = ConvBlock(in_ch=1, out_ch=cnn_channels)

        self.n_mels = n_mels
        self.cnn_channels = cnn_channels
        rep_size = (n_mels // 2) * cnn_channels
        self.fc = nn.Linear(rep_size, lstm_hidden)
        self.lstm = nn.LSTM(input_size=lstm_hidden, hidden_size=lstm_hidden, num_layers=lstm_layers,
                            bidirectional=True, batch_first=True)
        self.classifier = nn.Linear(lstm_hidden * 2, n_classes)

    def forward(self, x):

        x = self.conv(x)
        x = x.permute(0, 3, 1, 2)
        B, Tprime, C, F = x.shape
        x = x.reshape(B, Tprime, C * F)
        x = self.fc(x)
        x, _ = self.lstm(x)
        x = self.classifier(x)
        x = x.permute(1, 0, 2)
        return x

n_classes = len(vocab)
model = CNNRNN_ASR(n_mels=N_MELS, cnn_channels=128, lstm_hidden=512, lstm_layers=2, n_classes=n_classes)
model.to(device)
print("Model params:", sum(p.numel() for p in model.parameters())/1e6, "M")


Model params: 13.338047 M


In [24]:

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(meta, test_size=0.1, random_state=42, stratify=None)
train_ds = BengaliASRDataset(train_df.reset_index(drop=True))
val_ds = BengaliASRDataset(val_df.reset_index(drop=True))

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=8, shuffle=False, collate_fn=collate_fn, num_workers=2)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
ctc_loss = nn.CTCLoss(blank=0, zero_infinity=True)
scaler = GradScaler()

def greedy_decode(log_probs, input_lengths):

    probs = torch.argmax(log_probs, dim=2)
    T, B = probs.shape
    out_texts = []
    probs = probs.cpu().numpy().T
    for b in range(B):
        seq = probs[b].tolist()

        prev = -1
        chars = []
        for p in seq:
            if p != prev and p != 0:
                chars.append(idx2char.get(p, ""))
            prev = p
        out_texts.append("".join(chars))
    return out_texts

best_wer = 1.0
EPOCHS = 12

for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    model.train()
    running_loss = 0.0
    for batch_idx, (feats_padded, feat_lens, targets_concat, target_lens, targets_list) in enumerate(train_loader):
        feats_padded = feats_padded.to(device)

        input_lengths = (feat_lens // 2).to(device)
        optimizer.zero_grad()
        with autocast():
            logits = model(feats_padded)
            log_probs = F.log_softmax(logits, dim=2)

            loss = ctc_loss(log_probs, targets_concat.to(device), input_lengths, target_lens.to(device))
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item()
        if (batch_idx+1) % 50 == 0:
            print(f"Epoch {epoch} Batch {batch_idx+1}: avg loss {running_loss/(batch_idx+1):.4f}")
    avg_train_loss = running_loss / (batch_idx+1)

model.eval()
all_refs, all_hyps = [], []
with torch.no_grad():
    for feats_padded, feat_lens_val, targets_concat_val, target_lens_val, targets_list_val in val_loader:
        feats_padded = feats_padded.to(device)
        input_lengths_val = (feat_lens_val // 2).to(device)
        logits = model(feats_padded)
        log_probs = F.log_softmax(logits, dim=2)

        hyps = greedy_decode(log_probs, input_lengths_val)


        for tgt in targets_list_val:
            ref = "".join([idx2char[int(i)] for i in tgt.numpy().tolist() if int(i) != 0])
            all_refs.append(ref)
        all_hyps.extend(hyps)


    computed_wer = wer(all_refs, all_hyps)
    print(f"Epoch {epoch} finished in {time.time()-t0:.1f}s — TrainLoss: {avg_train_loss:.4f} — Val WER: {computed_wer:.4f}")

    if computed_wer < best_wer:
        best_wer = computed_wer
        ckpt_path = os.path.join(MODEL_DIR, f"best_asr_epoch{epoch}_wer{best_wer:.4f}.pt")
        torch.save(model.state_dict(), ckpt_path)
        print("Saved best model to", ckpt_path)


/tmp/ipython-input-1350547613.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipython-input-1350547613.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1 Batch 50: avg loss 3.3542
Epoch 1 Batch 100: avg loss 3.3534
Epoch 1 Batch 150: avg loss 3.3509
Epoch 1 Batch 200: avg loss 3.3466
Epoch 1 Batch 250: avg loss 3.3316
Epoch 1 Batch 300: avg loss 3.2634
Epoch 2 Batch 50: avg loss 2.4235
Epoch 2 Batch 100: avg loss 2.3186
Epoch 2 Batch 150: avg loss 2.2233
Epoch 2 Batch 200: avg loss 2.1505
Epoch 2 Batch 250: avg loss 2.0855
Epoch 2 Batch 300: avg loss 2.0206
Epoch 3 Batch 50: avg loss 1.5783
Epoch 3 Batch 100: avg loss 1.5523
Epoch 3 Batch 150: avg loss 1.5316
Epoch 3 Batch 200: avg loss 1.5102
Epoch 3 Batch 250: avg loss 1.4881
Epoch 3 Batch 300: avg loss 1.4694
Epoch 4 Batch 50: avg loss 1.2541
Epoch 4 Batch 100: avg loss 1.2396
Epoch 4 Batch 150: avg loss 1.2355
Epoch 4 Batch 200: avg loss 1.2282
Epoch 4 Batch 250: avg loss 1.2145
Epoch 4 Batch 300: avg loss 1.2061
Epoch 5 Batch 50: avg loss 1.0653
Epoch 5 Batch 100: avg loss 1.0559
Epoch 5 Batch 150: avg loss 1.0415
Epoch 5 Batch 200: avg loss 1.0467
Epoch 5 Batch 250: avg lo

In [19]:

all_text = " ".join(meta['transcript'].astype(str).tolist())
chars = sorted(list(set(all_text)))

vocab = ["<blank>"] + chars
char2idx = {c:i for i,c in enumerate(vocab)}
idx2char = {i:c for c,i in char2idx.items()}

print("Vocab size (including blank):", len(vocab))
print("Example tokens:", list(vocab)[:30])

def encode_text(text):

    return [char2idx.get(ch, 0) for ch in text]

def decode_indices(indices):

    s = []
    prev = None
    for idx in indices:
        if idx != prev and idx != 0:
            s.append(idx2char.get(idx, ""))
        prev = idx
    return "".join(s)


Vocab size (including blank): 63
Example tokens: ['<blank>', ' ', '।', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ']


In [25]:

model.load_state_dict(torch.load("/content/models/best_asr_epoch12_wer0.7843.pt", map_location=device))
model.eval()

def transcribe_file(audio_path):
    wav, sr = sf.read(audio_path)
    if sr != 16000:
        wav = librosa.resample(wav.astype(np.float32), orig_sr=sr, target_sr=16000)
    feat = compute_log_mel(wav)
    x = torch.from_numpy(feat).unsqueeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        log_probs = F.log_softmax(logits, dim=2)
        preds = torch.argmax(log_probs, dim=2).squeeze(1).cpu().numpy().tolist()
    return decode_indices(preds)


sample_path = meta.iloc[10]["audio_filepath"]
print(" Reference:", meta.iloc[10]["transcript"])
print(" Predicted:", transcribe_file(sample_path))


🎧 Reference: গত একত্রিশ জানুয়ারি সকালে চট্টগ্রাম মহানগরের চান্দগাঁও আবাসিক এলাকার একটি বাসায় ডা
🗣️ Predicted: গত একত্রিশ জানুয়ারি সাকালে চট্টগ্রাম মহানগরের চান্দগাঁও আবাসিক এলাকার একটি বাসায় ডা


In [26]:
model.eval()
example = torch.randn(1,1,N_MELS,300).to(device)
traced = torch.jit.trace(model.to('cpu'), example.cpu())
torch.jit.save(traced, "/content/models/asr_cnnrnn_torchscript.pt")
print(" TorchScript model saved for deployment!")


✅ TorchScript model saved for deployment!


In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.jit.load("/content/models/asr_cnnrnn_torchscript.pt", map_location=device)
model.to(device)
model.eval()

# Force all submodules to device
for name, module in model.named_modules():
    for param in module.parameters(recurse=False):
        param.data = param.data.to(device)
    for buffer in module.buffers(recurse=False):
        buffer.data = buffer.data.to(device)

print(f" Model and all LSTM weights moved to: {device}")


✅ Model and all LSTM weights moved to: cuda


In [41]:
import torch, numpy as np, librosa, soundfile as sf, torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.jit.load("/content/models/asr_cnnrnn_torchscript.pt", map_location=device)
model.to(device)
model.eval()


for name, module in model.named_modules():
    for param in module.parameters(recurse=False):
        param.data = param.data.to(device)
    for buffer in module.buffers(recurse=False):
        buffer.data = buffer.data.to(device)

print(f" Model loaded and synced on: {device}")

def transcribe_file(audio_path):
    wav, sr = sf.read(audio_path)
    if len(wav.shape) > 1:
        wav = np.mean(wav, axis=1)
    if sr != 16000:
        wav = librosa.resample(wav.astype(np.float32), orig_sr=sr, target_sr=16000)
    feat = compute_log_mel(wav)
    x = torch.from_numpy(feat).unsqueeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        log_probs = F.log_softmax(logits, dim=2)
        preds = torch.argmax(log_probs, dim=2).squeeze(1).cpu().numpy().tolist()
    return decode_indices(preds)


✅ Model loaded and synced on: cuda


In [46]:

model.eval()
scripted = torch.jit.script(model)
torch.jit.save(scripted, "/content/models/asr_cnnrnn_torchscript.pt")
print(" Scripted model saved without device lock")


✅ Scripted model saved without device lock


In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.jit.load("/content/models/asr_cnnrnn_torchscript.pt", map_location=device)
model.to(device).eval()
print("✅ Scripted model loaded on", device)


✅ Scripted model loaded on cuda


In [48]:
model.eval()
scripted = torch.jit.script(model)
torch.jit.save(scripted, "/content/models/asr_cnnrnn_torchscript.pt")


In [51]:

model.eval()

scripted = torch.jit.script(model)
torch.jit.save(scripted, "/content/models/asr_cnnrnn_scripted_fixed.pt")

print("✅ Saved SCRIPTED model at /content/models/asr_cnnrnn_scripted_fixed.pt")


✅ Saved SCRIPTED model at /content/models/asr_cnnrnn_scripted_fixed.pt


In [52]:
# Load your trained model (TorchScript preferred)


import torch, torch.nn.functional as F
import numpy as np, librosa, soundfile as sf

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)


MODEL_PATH = "/content/models/asr_cnnrnn_scripted_fixed.pt"

model = torch.jit.load(MODEL_PATH, map_location=device)
model.to(device)
model.eval()

print("Scripted model loaded successfully on", device)


✅ Using device: cuda
✅ Scripted model loaded successfully on cuda


In [54]:

model.eval()

scripted = torch.jit.script(model)
torch.jit.save(scripted, "/content/models/asr_cnnrnn_scripted_fixed.pt")

print("✅ Scripted model saved at /content/models/asr_cnnrnn_scripted_fixed.pt")


✅ Scripted model saved at /content/models/asr_cnnrnn_scripted_fixed.pt


In [55]:

#   Fixed model-loading cell  (uses SCRIPTED model)

import torch, torch.nn.functional as F
import numpy as np, librosa, soundfile as sf

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

MODEL_PATH = "/content/models/asr_cnnrnn_scripted_fixed.pt"   # <-- new scripted model

model = torch.jit.load(MODEL_PATH, map_location=device)
model.to(device)
model.eval()
print("✅ Scripted model loaded successfully on", device)


✅ Using device: cuda
✅ Scripted model loaded successfully on cuda


In [56]:
sample = meta.iloc[0]["audio_filepath"]
print("Ref:", meta.iloc[0]["transcript"])
print("Pred:", transcribe_file(sample))


Ref: শেখ হাসিনাকে হত্যাচেষ্টা মামলায় এগার জনের বিশ বছর করে কারাদণ্ড
Pred: ❌ Error during transcription: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__.py", line 24, in forward
    F = ops.prim.NumToTensor(torch.size(x0, 3))
    input = torch.reshape(x0, [_0, _1, int(torch.mul(C, F))])
    _2 = (lstm).forward((fc).forward(input, ), )
          ~~~~~~~~~~~~~ <--- HERE
    _3 = torch.permute((classifier).forward(_2, ), [1, 0, 2])
    return _3
  File "code/__torch__/torch/nn/modules/rnn.py", line 46, in forward
    _1 = [hx, hx0]
    _2 = [weight_ih_l0, weight_hh_l0, bias_ih_l0, bias_hh_l0, weight_ih_l0_reverse, weight_hh_l0_reverse, bias_ih_l0_reverse, bias_hh_l0_reverse, weight_ih_l1, weight_hh_l1, bias_ih_l1, bias_hh_l1, weight_ih_l1_reverse, weight_hh_l1_reverse, bias_ih_l1_reverse, bias_hh_l1_reverse]
    input, _3, _4 = torch.lstm(argument_1, _1, _2, True, 2, 0., False, True, T

In [49]:

#  Bengali Speech-to-Text Web App  (CNN-RNN Hybrid)
# Safe for GPU, works in Google Colab


!pip install --quiet gradio torch torchvision torchaudio librosa soundfile jiwer

import gradio as gr
import torch, numpy as np, librosa, soundfile as sf, torch.nn.functional as F


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)




for name, module in model.named_modules():
    for param in module.parameters(recurse=False):
        param.data = param.data.to(device)
    for buffer in module.buffers(recurse=False):
        buffer.data = buffer.data.to(device)

print(f"✅ Model loaded & synced on {device}")



SAMPLE_RATE = 16000
N_MELS = 80
N_FFT = 1024
HOP_LENGTH = 256
WIN_LENGTH = 1024

def compute_log_mel(wav):
    mel = librosa.feature.melspectrogram(
        y=wav, sr=SAMPLE_RATE, n_fft=N_FFT,
        hop_length=HOP_LENGTH, win_length=WIN_LENGTH,
        n_mels=N_MELS, power=1.0
    )
    log_mel = librosa.power_to_db(mel)
    log_mel = (log_mel - log_mel.mean()) / (log_mel.std() + 1e-9)
    return log_mel.astype(np.float32)


def decode_indices(indices):
    s, prev = [], None
    for idx in indices:
        if idx != prev and idx != 0:
            s.append(idx2char.get(idx, ""))
        prev = idx
    return "".join(s)


def transcribe_file(audio_path):
    try:
        wav, sr = sf.read(audio_path)
        if len(wav.shape) > 1:
            wav = np.mean(wav, axis=1)
        if sr != SAMPLE_RATE:
            wav = librosa.resample(wav.astype(np.float32), orig_sr=sr, target_sr=SAMPLE_RATE)

        feat = compute_log_mel(wav)
        x = torch.from_numpy(feat).unsqueeze(0).unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model(x)
            log_probs = F.log_softmax(logits, dim=2)
            preds = torch.argmax(log_probs, dim=2).squeeze(1).cpu().numpy().tolist()

        text = decode_indices(preds)
        return text or "⚠️ Could not decode (try clearer/longer audio)."

    except Exception as e:
        return f"❌ Error during transcription: {e}"


def transcribe_gradio(audio):
    return transcribe_file(audio)

demo = gr.Interface(
    fn=transcribe_gradio,
    inputs=gr.Audio(sources=["microphone", "upload"], type="filepath", label="🎙️ Speak or upload Bengali audio"),
    outputs=gr.Textbox(label="📝 Transcribed Text"),
    title="🇧🇩 Bengali Speech-to-Text (CNN-RNN Hybrid)",
    description="Deep-learning Speech-to-Text system trained on Bhashini Bengali corpus using CNN + BiLSTM + CTC.",
    allow_flagging="never"
)

demo.launch(debug=True, share=True)


✅ Using device: cuda
✅ Model loaded & synced on cuda


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2a04b26739920d91dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://2a04b26739920d91dc.gradio.live


In [57]:


!pip install --quiet gradio torch torchvision torchaudio librosa soundfile jiwer

import torch, torch.nn as nn, torch.nn.functional as F
import numpy as np, librosa, soundfile as sf, gradio as gr
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)


class ConvBlock(nn.Module):
    def __init__(self, in_ch=1, out_ch=128):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(out_ch)
        self.pool = nn.MaxPool2d((2,2))
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        return x

class CNNRNN_ASR(nn.Module):
    def __init__(self, n_mels=80, cnn_channels=128, lstm_hidden=512, lstm_layers=2, n_classes=100):
        super().__init__()
        self.conv = ConvBlock(1, cnn_channels)
        rep_size = (n_mels // 2) * cnn_channels
        self.fc = nn.Linear(rep_size, lstm_hidden)
        self.lstm = nn.LSTM(lstm_hidden, lstm_hidden, lstm_layers,
                            bidirectional=True, batch_first=True)
        self.classifier = nn.Linear(lstm_hidden * 2, n_classes)
    def forward(self, x):

        x = self.conv(x)
        x = x.permute(0,3,1,2).reshape(x.size(0), x.size(3), -1)
        x = self.fc(x)
        h0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size, device=x.device)
        c0 = torch.zeros_like(h0)
        x, _ = self.lstm(x, (h0, c0))
        x = self.classifier(x)
        x = x.permute(1,0,2)
        return x


MODEL_PATH = "/content/models/best_asr_epoch12_wer0.7843.pt"


try:
    len(vocab)
except NameError:
    meta = pd.read_csv("/content/dataset/Kathbath-Bengali-Test-Known/metadata.csv")
    all_text = " ".join(meta['transcript'].astype(str).tolist())
    chars = sorted(list(set(all_text)))
    vocab = ["<blank>"] + chars
    char2idx = {c:i for i,c in enumerate(vocab)}
    idx2char = {i:c for c,i in char2idx.items()}
    print("✅ Vocab rebuilt with size:", len(vocab))

model = CNNRNN_ASR(n_mels=80, cnn_channels=128, lstm_hidden=512,
                   lstm_layers=2, n_classes=len(vocab))
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device).eval()
print("✅ Model loaded successfully from checkpoint!")


SAMPLE_RATE = 16000
N_MELS = 80
N_FFT = 1024
HOP_LENGTH = 256
WIN_LENGTH = 1024

def compute_log_mel(wav):
    mel = librosa.feature.melspectrogram(y=wav, sr=SAMPLE_RATE, n_fft=N_FFT,
                                         hop_length=HOP_LENGTH, win_length=WIN_LENGTH,
                                         n_mels=N_MELS, power=1.0)
    log_mel = librosa.power_to_db(mel)
    log_mel = (log_mel - log_mel.mean()) / (log_mel.std() + 1e-9)
    return log_mel.astype(np.float32)

def decode_indices(indices):
    s, prev = [], None
    for idx in indices:
        if idx != prev and idx != 0:
            s.append(idx2char.get(idx, ""))
        prev = idx
    return "".join(s)


def transcribe_file(audio_path):
    try:
        wav, sr = sf.read(audio_path)
        if len(wav.shape) > 1: wav = np.mean(wav, axis=1)
        if sr != SAMPLE_RATE:
            wav = librosa.resample(wav.astype(np.float32), orig_sr=sr, target_sr=SAMPLE_RATE)
        feat = compute_log_mel(wav)
        x = torch.from_numpy(feat).unsqueeze(0).unsqueeze(0).to(device)
        with torch.no_grad():
            logits = model(x)
            log_probs = F.log_softmax(logits, dim=2)
            preds = torch.argmax(log_probs, dim=2).squeeze(1).cpu().numpy().tolist()
        text = decode_indices(preds)
        return text or "⚠️ Could not decode (try clearer/longer audio)."
    except Exception as e:
        return f"❌ Error during transcription: {e}"


def transcribe_gradio(audio):
    return transcribe_file(audio)

demo = gr.Interface(
    fn=transcribe_gradio,
    inputs=gr.Audio(sources=["microphone", "upload"], type="filepath", label="🎙️ Speak or upload Bengali audio"),
    outputs=gr.Textbox(label="📝 Transcribed Text"),
    title="🇧🇩 Bengali Speech-to-Text (CNN-RNN Hybrid Deep Model)",
    description="Speech-to-Text model trained on Bhashini Bengali corpus using CNN+BiLSTM+CTC. GPU-safe & permanent version.",
    allow_flagging="never"
)

demo.launch(debug=True, share=True)


✅ Using device: cuda
✅ Model loaded successfully from checkpoint!


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6e4f7062c2356a21da.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://6e4f7062c2356a21da.gradio.live


In [61]:

#  Bengali Speech → Hindi Text (No Conflicts Version)


!pip install --quiet deep-translator

from deep_translator import GoogleTranslator

def transcribe_gradio(audio):
    bengali_text = transcribe_file(audio)
    if bengali_text.startswith("❌"):
        return bengali_text
    try:
        translated = GoogleTranslator(source='bn', target='hi').translate(bengali_text)
        return f"🗣️ Bengali: {bengali_text}\n🇮🇳 Hindi: {translated}"
    except Exception as e:
        return f"⚠️ Translation failed: {e}\nBengali text: {bengali_text}"

demo = gr.Interface(
    fn=transcribe_gradio,
    inputs=gr.Audio(sources=["microphone", "upload"], type="filepath", label="🎙️ Speak or upload Bengali audio"),
    outputs=gr.Textbox(label="📝 Hindi Transcription"),
    title="🎧 Bengali Speech → Hindi Text Translator",
    description="CNN–RNN Speech-to-Text for Bengali + automatic Hindi translation (deep-translator)",
    allow_flagging="never"
)

demo.launch(debug=True, share=True)


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2957271a882fcf3ac9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://2957271a882fcf3ac9.gradio.live
